> ユーザがアイテムに与えるレイティングは、通常、ユーザの好みを反映している。したがって、<strong style="color:blue;">アイテムの評価が似ている２人のユーザは嗜好が似ている可能性が高い</strong>。この直感にもとづいて、評価行動がユーザ$i$に類似する他のユーザの集合を得ることができる。

ユーザ$i$におけるアイテム$j$のスコアは、ユーザ$i$に類似しているユーザによってアイテム$j$に与えられたレイティングの平均を用いて得ることができる。このようなアプローチは、ユーザのアイテムに対する過去のレイティングのみに基づいてアイテムを好む程度を予測するため、ユーザまたはアイテムの素性に依存しない。

In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 日本語フォントを設定
font = {'family': 'IPAexGothic'}
mpl.rc('font', **font)

%matplotlib inline

In [2]:
import os
import re
import sys


pwd = os.getcwd()
path = re.search('.+/推薦システム', pwd).group(0)

sys.path.append(path)
print(sys.path)

['', '/Users/taiyou/Desktop/machine-learning/推薦システム/notebook', '/Users/taiyou/.pyenv/versions/anaconda3-5.3.1/lib/python37.zip', '/Users/taiyou/.pyenv/versions/anaconda3-5.3.1/lib/python3.7', '/Users/taiyou/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/lib-dynload', '/Users/taiyou/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages', '/Users/taiyou/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/aeosa', '/Users/taiyou/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/algotrader-0.0.1-py3.7.egg', '/Users/taiyou/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions', '/Users/taiyou/.ipython', '/Users/taiyou/Desktop/machine-learning/推薦システム']


# 準備

In [3]:
data_org = pd.read_csv(
   'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 
    names=["user_id", "item_id", "rating", "timestamp"], 
    sep="\t"
)
data_org.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
from datetime import datetime

data_org['timestamp'] = data_org['timestamp'].map(datetime.fromtimestamp)

In [5]:
data_org = data_org.set_index(['user_id', 'item_id'])

In [6]:
評価値_df = data_org.copy()
評価値_df.head()

,,rating,timestamp
user_id,item_id,,
196,242,3,1997-12-05 00:55:49
186,302,3,1998-04-05 04:22:22
22,377,1,1997-11-07 16:18:36
244,51,2,1997-11-27 14:02:03
166,346,1,1998-02-02 14:33:16


## 評価行列の作成

In [7]:
評価値行列_df = 評価値_df['rating'].unstack()
# 評価行列_df.fillna(0.0, inplace=True)
評価値行列_df.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---

# ユーザ間の類似度に基づいた手法

> まず、ユーザ$i$と似た他のユーザがアイテム$j$に与えた評価に基づいて、ユーザ$i$が評価していないアイテム$j$に与えるレイティング(またはスコア)$s_{ij}$を予測する協調フィルタリング手法から考える。

## ユーザベース協調フィルタリング
スコア関数の一般的な選択肢の１つとして、類似ユーザの平均レイティングが考えられる。また、ユーザ$i$に似ているユーザに大きな重みを割り当てる加重平均を用いることもできる。
ユーザ$i$のアイテム$j$に対する推定レイティング$s_{ij}$を記述すると以下のようになる。

$$
s_{ij} = {\bar {y}}_{i} + \frac { \sum _{ \ell \in {\bf I}_{j}\left(i\right) }{ w\left(i,\ell\right) \left( y_{\ell j} - {\bar {y}}_{\ell} \right) } }{ \sum _{ \ell \in {\bf I}_{j}\left(i\right) }{ \left| w\left(i,\ell\right) \right| } }
$$

> - ${\bf I}_{j}\left(i\right)$ : アイテム$j$を評価したユーザ$i$に類似するユーザの集合
> - $w\left(i,\ell\right)$ : ユーザ$i$がアイテム$j$を評価する際のユーザ$\ell$の評価に対する重み
> - $\bar {y}_{i}$ : ユーザ$i$の平均レイティング
> - $\left( y_{\ell j} - {\bar {y}}_{\ell} \right)$ : ユーザの個々の評価バイアス(評価値が同じであってもユーザ間で満足度が異なることがあるため)を軽減するために平均値を使用してレイティングを中心化している。
>     - 中心化に加えて、ユーザのレイティングの標準偏差で中心化されたレイティングを割ることによってさらに標準化することができる

## ユーザ間の類似度関数

ユーザ間の類似度関数の一般的な選択肢の１つは、ピアソン相関である。ここでユーザ$i$と$\ell$との間の類似度は以下のように定義される。

$$
sim\left( i, \ell \right) = \frac { \sum _{ j\in { J }_{ i\ell  } }{ \left( { y }_{ ij }-\bar { { y }_{ i } }  \right) \left( { y }_{ \ell j }-\bar { { y }_{ \ell  } }  \right)  }  }{ \sqrt { \sum _{ j\in { J }_{ i\ell  } }{ { \left( { y }_{ ij }-\bar { { y }_{ i } }  \right)  }^{ 2 } }  } \sqrt { \sum _{ j\in { J }_{ i\ell  } }{ { \left( { y }_{ \ell j }-\bar { { y }_{ \ell  } }  \right)  }^{ 2 } }  }  } 
$$

 - $J_{i \ell}$ : ユーザ$i$と$\ell$の両方によって評価されたアイテムの集合

## 近傍選択

類似ユーザ集合${\bf I}_{j}\left(i\right)$の構築方法はいくつかある。

> - <strong>単純にアイテム$j$を評価した全ユーザを考慮</strong>し、ユーザ$i$と$\ell$との間の類似性を重み$w\left(i, \ell\right)$と定義する
> - ユーザ$i$と類似度の高いユーザを上から$n$人選択する
> - ユーザ$i$との類似度が閾値よりも高いユーザを選択する

## 重み付け

最も一般的な重み付け方法は$w\left(i,\ell\right) = sim\left(i,\ell\right)$とすることである。

$sim\left(i,\ell\right)$がユーザ$i$と$\ell$の両方によって評価された小さなアイテム集合${\bf J}_{i\ell}$に基づいて計算されるとき、サンプルサイズが小さいため信頼性が低い可能性がある。サンプルサイズが小さい場合に生じる問題に対処する１つの方法として、信頼性が低い類似度の重みを小さくすることがあげられる。例えば、Herlocker et al.(1999)では、次式を用いている。

$$
w\left( i, \ell \right) = \min {\left\{ \frac {\left| {\bf J}_{i\ell} \right|}{\alpha}, 1 \right\}} \cdot sim\left( i,\ell \right)
$$

In [8]:
評価値行列_df.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
from domain.model.cf import UserBaseCF

ubcf = UserBaseCF()
ubcf.fit(評価値行列_df)

In [43]:
ubcf.predict(1, 2)

3.3447518415881716

In [11]:
from tqdm import tqdm

score_dict = {}
for i in tqdm(評価値行列_df.index[:5]):
    score_dict[i] = [ubcf.predict(i, j) for j in 評価値行列_df.columns]

  0%|          | 0/5 [00:00<?, ?it/s]/Users/taiyou/Desktop/machine-learning/推薦システム/domain/model/cf/user_base_cf.py:51: RuntimeWarning: invalid value encountered in double_scalars
  (weight_ser * (y_lj - y_l)).sum() / weight_ser.abs().sum()
100%|██████████| 5/5 [00:44<00:00,  8.86s/it]


In [12]:
pd.DataFrame(score_dict).T

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
1,3.966871,3.344752,3.245953,3.673835,3.265477,3.602586,3.942484,3.759416,4.115454,3.729687,...,4.757353,3.473238,3.150964,2.150964,3.398313,1.488799,3.488799,2.488799,3.629631,3.244364
2,3.882326,3.444407,3.290846,3.673374,3.495695,4.007114,3.946685,3.969174,4.038008,3.573015,...,2.562619,NaN,3.250347,2.250347,3.497696,5.831173,3.831173,4.831173,3.729014,3.343747
3,2.870299,2.572094,2.573143,2.737020,2.681179,2.591166,2.909221,2.925830,3.024487,2.711515,...,1.649237,2.933352,2.336966,1.336966,3.008278,4.917792,2.917792,3.917792,2.815633,NaN
4,4.463890,4.189383,4.082123,4.282544,4.190392,4.597468,4.442616,4.472127,4.352374,4.560379,...,5.480392,4.196277,3.874003,2.874003,4.121352,6.454829,4.454829,5.454829,4.352670,3.967403
5,3.170103,2.621060,2.450848,2.933294,2.631367,3.098643,3.206504,3.168473,3.111540,2.968731,...,4.021345,2.737230,2.414956,1.414956,2.662304,NaN,NaN,NaN,2.893623,2.508355


In [23]:
ユーザID = 1
未知のアイテムID = 評価値行列_df.loc[ユーザID].index[評価値行列_df.loc[ユーザID].isnull()]
推定レイティング = pd.Series([ubcf.predict(i, j) for j in 未知のアイテムID], index=未知のアイテムID).sort_values(ascending=False)

/Users/taiyou/Desktop/machine-learning/推薦システム/domain/model/cf/user_base_cf.py:51: RuntimeWarning: invalid value encountered in double_scalars
  (weight_ser * (y_lj - y_l)).sum() / weight_ser.abs().sum()


In [24]:
推定レイティング.head()

item_id
1309    5.251097
1308    4.812193
814     4.776801
1463    4.751369
1536    4.578912
dtype: float64

In [29]:
推定レイティング[推定レイティング.notnull()].tail()

item_id
1502    0.234547
1618    0.157305
1621    0.071891
599    -0.090975
1659   -0.180322
dtype: float64

---

# アイテム間の類似度に基づいた手法

## アイテムベース協調フィルタリング
ユーザ$i$のアイテム$j$に対するレイティングを予測する
$$
s_{ij} = {\bar {y}}_{j} + \frac { \sum _{ \ell \in {\bf I}_{i}\left(j\right) }{ w\left(j,\ell\right) \left( y_{i \ell} - {\bar {y}}_{\ell} \right) } }{ \sum _{ \ell \in {\bf I}_{i}\left(j\right) }{ \left| w\left(j,\ell\right) \right| } }
$$

 - ${\bf I}_{i}\left(j\right)$ : ユーザ$i$によって評価されたアイテム$j$と似ているアイテムの集合
 - $w\left(i,\ell\right)$ : アイテム$j$に対するユーザ$i$の評価を予測するために、ユーザ$i$がアイテム$\ell$に与えたレイティングに割り当てる重み
 - $\bar {y}_{j}$ : アイテム$j$の平均レイティング

---

# 行列分解

WIP

---

# 例)どんぶり専門店『丼兵衛』

## 評価値行列

 - $\boldsymbol {\chi} = \left\{ 1,\dots, 4 \right\}$
 - $\boldsymbol {y} = \left\{ 1,\dots,4 \right\}$
 - 3段階の採点法 : $$

In [31]:
R_arr = np.array([
    [1, 3, np.nan,  3],
    [np.nan, 1, 3, np.nan],
    [2, 1, 3, 1],
    [1, 3, 2, np.nan]
])

R_df = pd.DataFrame(
    R_arr, 
    columns=['親子丼', '牛丼', '海鮮丼', 'カツ丼'], 
    index=['山田', '田中', '佐藤', '鈴木']
)
R_df.head()

,親子丼,牛丼,海鮮丼,カツ丼
山田,1.0,3.0,NaN,3.0
田中,NaN,1.0,3.0,NaN
佐藤,2.0,1.0,3.0,1.0
鈴木,1.0,3.0,2.0,NaN


上記の評価値行列から、「**田中$x=2$**」の「**親子丼$y = 1$**」への推定評価値$\hat {r}_{2,1}$を求める

In [41]:
from domain.model.cf import UserBaseCF

rating_estimator = UserBaseCF()
rating_estimator.fit(R_df)
rating_estimator.predict('田中', '親子丼')

2.625

この値は、最大値3にかなり近く、「田中」は「親子丼」が好きであると予測される